<h1 align=center><font size = 5>Capstone : Segmenting and Clustering Neighborhoods in Toronto City</font></h1>

### Part 2 :  Assign Geographical Coordinates

## Introduction

This part of the project is intend:  
- Get a list of neighborhoods in Toronto from Wikipedia page and clean the data    
- Get latitude and longitude of each neighborhood.   


First, import the nessary library, get and transform data to a dataframe from wikipedia

In [7]:
# import necessary package
import pandas as pd # library to handle data in a vectorized manner
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
import numpy as np  # library for data analsysis
import requests   # library to handle requests
from bs4 import BeautifulSoup # for pulling data out of HTML and XML files
print('Libraries imported.')

Libraries imported.


## 1. Get the Data

Read the list of neighborhoods by scraping the wikepedia page

In [9]:
# collect web page with Request
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M") # res is assigned as Response object
soup = BeautifulSoup(res.content,'lxml') # BeautifulSoup object
table = soup.find_all('table')[0] # extract tag from a page get and get the first table
canada_postal = pd.read_html(str(table), skiprows=1)[0] # Return a list of dataframe
canada_postal.columns=['PostalCode','Borough','Neighborhood'] # add columns name
canada_postal=canada_postal[canada_postal.Borough != 'Not assigned'] #  filter cells with a borough that is Not assigned
print(canada_postal.shape)
canada_postal.head()


(210, 3)


,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


Construct a new dataframe by group the dataframe by Postcode and Borough and concatenate
 all neighborhoods into comma separated list

In [10]:
canada_postal_g =canada_postal.groupby(['PostalCode', 'Borough'])['Neighborhood'].\
agg([('Neighborhood', ','.join)]).reset_index()
print(canada_postal_g.shape)

canada_postal_g.head()


(103, 3)


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [11]:
# check borough with label "Not assigned"
canada_postal.loc[canada_postal['Neighborhood'] == "Not assigned"]

,PostalCode,Borough,Neighborhood
9,M9A,Queen's Park,Not assigned


In [12]:
# change borough name with "not assigned" to the name of the Neighborhhod
canada_postal['Neighborhood'] = np.where((canada_postal.Neighborhood=='Not assigned'),
                                       canada_postal.Borough, canada_postal.Neighborhood)
canada_postal.loc[canada_postal['Neighborhood'] == "Queen's Park"]

,PostalCode,Borough,Neighborhood
7,M7A,Downtown Toronto,Queen's Park
9,M9A,Queen's Park,Queen's Park


In [8]:
canada_postal.shape

(210, 3)

In [9]:
canada_postal_g.shape

(103, 3)

## Get geographical coordiante

In order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood. Unfortunatly, I was not able get the expect resut using **geocoder**. So, I prefer to use the csv file provided and merge with Toronto Borough

In [14]:
#downloading dataset
df = pd.read_csv("Geospatial_Coordinates.csv")

df.columns=['PostalCode','Latitude','Longitude']
df.head(10)


Toronto = pd.merge(canada_postal_g,
                 df[['PostalCode','Latitude', 'Longitude']],
                 on='PostalCode')
Toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
